In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import pycountry
import warnings
import plotly.graph_objects as go
# from wordcloud import WordCloud, STOPWORDS
import matplotlib.dates as mdates
import re
import string
from collections import Counter
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.utils import shuffle

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Embedding
from keras.regularizers import l2

In [14]:
data = pd.read_csv('finalbully.csv')
data.head()

,Unnamed: 0,label,full_text
0,0,Offensive,So Drasko just said he was impressed the girls...
1,2,Offensive,Drasko they didn't cook half a bird you idiot ...
2,4,Offensive,Hopefully someone cooks Drasko in the next ep ...
3,6,Offensive,of course you were born in serbia...you're as ...
4,7,Offensive,These girls are the equivalent of the irritati...


In [15]:
def clean_text(text):
    
    #remove urls
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text = url_pattern.sub(r'', text)
    
    #remove html
    html_pattern = re.compile(r'<.*?>')
    text = html_pattern.sub(r'', text)
    
    #remove emojis
    emoji_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    text = emoji_pattern.sub(r'',text)
    
    #remove punctuations
    table = str.maketrans("", string.punctuation)
    text = text.translate(table)

    return text

In [19]:
import re
from bs4 import BeautifulSoup
from sklearn import linear_model
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase


def partition(x):
    if x == 'Offensive':
        return 0
    else:
        return 1

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])


def clean_text(sentance):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    return sentance.strip()




filtered_data= pd.read_csv('finalbully.csv')
# filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 LIMIT 10000""", con)
filtered_data['label'] = filtered_data['label'].map(partition)
# sorted_data = filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
final = filtered_data.drop_duplicates(subset={"full_text"}, keep='first', inplace=False)
# final = final.sort_values('Time', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
# final = final[final.HelpfulnessNumerator <= final.HelpfulnessDenominator]


preprocessed_reviews = []
for sentence in final['full_text'].values:
    pint(sentence)
    preprocessed_reviews.append(clean_text(sentence))

count_vect = CountVectorizer()
count_vect.fit(preprocessed_reviews)
joblib.dump(count_vect, 'count_vectcontextbased.pkl')
X = count_vect.transform(preprocessed_reviews)
print(X.shape)
Y = final['label'].values

(35744, 37803)


In [22]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X,Y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [23]:
y_pred = classifier.predict(X)
y_pred

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [27]:
from sklearn.metrics import confusion_matrix,classification_report
cm = confusion_matrix(Y, y_pred)

In [28]:
cm

array([[23526,     5],
       [    2, 12211]], dtype=int64)

In [30]:
classification_report(Y,y_pred)

'              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00     23531\n           1       1.00      1.00      1.00     12213\n\n    accuracy                           1.00     35744\n   macro avg       1.00      1.00      1.00     35744\nweighted avg       1.00      1.00      1.00     35744\n'

In [32]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X,Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [33]:
rfc_pred = rfc.predict(X)

In [34]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y, rfc_pred)

In [36]:
cm

array([[23524,     7],
       [    1, 12212]], dtype=int64)

In [37]:
classification_report(Y,rfc_pred)

'              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00     23531\n           1       1.00      1.00      1.00     12213\n\n    accuracy                           1.00     35744\n   macro avg       1.00      1.00      1.00     35744\nweighted avg       1.00      1.00      1.00     35744\n'